In [12]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib qt
import pandas as pd
from scipy.constants import pi
from scipy.optimize import curve_fit
plt.rcParams['font.size'] = 20

In [13]:
data = pd.read_csv("data.csv")
print(data)


    thete  coincident  time      rate  error coin  error rate       rel  \
0     -90         500   192  2.604167   22.360680    0.116462  4.472136   
1     -80         428   181  2.364641   20.688161    0.114299  4.833682   
2     -70         422   160  2.637500   20.542639    0.128391  4.867924   
3     -60         442   170  2.600000   21.023796    0.123669  4.756515   
4     -50         418   176  2.375000   20.445048    0.116165  4.891160   
5     -40         426   159  2.679245   20.639767    0.129810  4.845016   
6     -35         422   182  2.318681   20.542639    0.112872  4.867924   
7     -30         482   162  2.975309   21.954498    0.135522  4.554875   
8     -25         484   168  2.880952   22.000000    0.130952  4.545455   
9     -20         486   171  2.842105   22.045408    0.128921  4.536092   
10    -15         472   147  3.210884   21.725561    0.147793  4.602873   
11    -10         506   171  2.959064   22.494444    0.131546  4.445542   
12     -5         466   1

In [14]:
data = data.to_numpy()
theta = data[:, 0]
N_coin = data[:, 1]
t_coin = data[:, 2]
N_1 = data[:, 7]
t_1 = data[:, 8]
theta *= np.pi/180

In [15]:
tot_rate = (N_coin*t_1)/(N_1*t_coin)
N_coin_err = np.sqrt(N_coin)
N_1_err = np.sqrt(N_1)
tot_rate_err = np.sqrt(
                        ( (N_coin_err*t_1)/(N_1*t_coin) )**2 + ( (N_coin*t_1*N_1_err)/(t_coin*N_1**2) )**2 
                        )


In [16]:
def fit_func(x, a0, a1, a2, scale):
    return scale*(a0 + a1 * np.cos(x)**2 + a2 * np.cos(x)**4)

In [17]:
params_optimum, params_cov = curve_fit(fit_func, theta, tot_rate, sigma=tot_rate_err, p0=[1, 1/8, 1/24, 0.0023])

In [18]:
a0, a1, a2, scale = params_optimum
a0_err, a1_err, a2_err, scale_err = np.diag(params_cov)

In [19]:
print(params_optimum)
print(np.diag(params_cov))
print(np.diag(params_cov)/params_optimum * 100, '%')

[ 0.99843926 -0.0778094   0.19362633  0.00217909]
[4.94456231e+12 3.00295534e+10 1.85957377e+11 2.35523474e+07]
[ 4.95229156e+14 -3.85937332e+13  9.60393022e+13  1.08083493e+12] %


In [20]:
plt.errorbar(theta, tot_rate, yerr=tot_rate_err, ls="None", marker='.', ms=20, label='Data', elinewidth=3)
x = np.linspace(-pi/2, pi/2, 100)
plt.plot(x, fit_func(x, 1, 1/8, 1/24, scale), label='Expected', lw=3)
plt.plot(x, fit_func(x, *params_optimum), label='Fit', lw=3)
plt.legend(loc='best')